# Maestria en Explotacion de datos y Descubrimiento de conocimiento
### Sistemas de información geografica
### Trabajo Practico N°2

# Clasificación por pixel

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

In [3]:
import plot
import sql
import numpy as np
from sklearn.model_selection import train_test_split

## Constantes globales

In [4]:
RASTERS_PATH = "../datasets/images"
DATA_PATH    = "../datasets/data"
RESULT_PATH  = "../results"

In [5]:
DATES = [
    '2020-10-01', 
    '2020-11-01', 
    '2020-12-01',
    '2021-01-01', 
    '2021-02-20', 
    '2021-03-17'
]
RASTER_FILES = [
    '0000000000-0000000000', 
    '0000000000-0000012544'
]

# VERDAD_CAMPO = 'verdad_campo'
VERDAD_CAMPO = 'verdad_campo_aumentada'

## Funciones helper

In [6]:
flatten = lambda list: np.ndarray.flatten(np.array(list))
raster_path      = lambda path: f'{RASTERS_PATH}/{path}.tif'
raster_date_path = lambda date, file: raster_path(f'{date}/{file}')

data_path        = lambda file: f'{DATA_PATH}/{file}.shp'
result_path      = lambda file: f'{RESULT_PATH}/{file}'


def print_title(title): print(f'\n\n{title}...\n')

def class_statistics(
    raster_paths,
    labels_file  = VERDAD_CAMPO, 
    label_column = 'id', 
    out_file      = 'class_statistics.xml',
    verbose      = 0
):
    
    in_paths = " ".join(raster_paths)
    vec_path = data_path(labels_file)
    out_path = result_path(out_file)
    
    if verbose > 0:
        print_title('Generate class statistics')
        print(f'- In Paths: {in_paths}')
        print(f'- Vec Path: {vec_path}')
        print(f'- Field...: {label_column}')
        print(f'- Out Path: {out_path}\n\n')
    
    !time otbcli_PolygonClassStatistics -in {in_paths} -vec {vec_path} -field {label_column} -out {out_path} > /dev/null
    !head {out_path}
    return out_path


def merge_rasters(source_paths, target_path, verbose = 1, plot=0):
    !time gdal_merge.py -o {target_path} -of gtiff {" ".join(source_paths)}

    if verbose > 0:
        print_title('Merge Rasters')
        for idx, path in enumerate(source_paths):
            print(f'Source path {idx+1}: {path}')
            if plot > 0:
                plot.plot_raster(path)

        print(f'Target path:\n  - {target_path}\n\n')
        if plot > 0:
            plot.plot_raster(target_path)
        

def layer_info(file):
    !ogrinfo -so {data_path(file)}


def layer_table_info(file, table):
    !ogrinfo -so {data_path(file)} {table}


def layer_query(file, query):
    !ogrinfo -dialect sqlite -sql "{query}" {data_path(file)}

Sampling de observaciones:

In [7]:
def sampling(
    raster_paths,
    class_stat_path,
    out_rates_path, 
    out_sql_path,
    labels_file  = VERDAD_CAMPO,
    label_column = 'id',
    strategy     = 'smallest',
    verbose      = 0
):
    raster_paths = " ".join(raster_paths)

    if verbose > 0:
        print_title('Sampling')
        print(f'- In Paths......: {raster_paths}')
        print(f'- Vec Path......: {data_path(labels_file)}')
        print(f'- Field.........: {label_column}')
        print(f'- Instats Path..: {class_stat_path}')
        print(f'- Strategy......: {strategy}')
        print(f'- Out Rates Path: {result_path(out_rates_path)}')
        print(f'- Out SQL Path..: {result_path(out_sql_path)}\n\n')

    !time otbcli_SampleSelection \
        -in       {raster_paths} \
        -vec      {data_path(labels_file)} \
        -instats  {class_stat_path} \
        -field    {label_column} \
        -strategy {strategy} \
        -outrates {result_path(out_rates_path)} \
        -out      {result_path(out_sql_path)}

def sample_extraction(
    raster_paths,
    out_sql_file,
    label_column = 'id',
    verbose      = 0
):
    if verbose > 0:
        print_title('Sample extraction')
        print(f'- In Paths......: {" ".join(raster_paths)}')
        print(f'- Vec SQL Path..: {result_path(out_sql_file)}')
        print(f'- Field.........: {label_column}\n\n')

    !time otbcli_SampleExtraction \
        -in                   {" ".join(raster_paths)} \
        -vec                  {result_path(out_sql_file)} \
        -field                {label_column} \
        -outfield             prefix \
        -outfield.prefix.name band_

Funciones de visualización:

Funciones para la clasificación:

In [8]:
def compute_raster_statistics(
    raster_paths,
    stat_file,
    verbose = 0
):
    output_path = result_path(stat_file)

    if verbose > 0:
        print_title('Compute rasters statistics')
        print(f'- Raster Paths: {" ".join(raster_paths)}')
        print(f'- Stats Path : {output_path}\n\n')

    !time otbcli_ComputeImagesStatistics -il {" ".join(raster_paths)} -out.xml {output_path}

    return output_path

In [9]:
def train_clasifier(
    sql_file,
    stat_file,
    features         = ['band_0', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6'],
    label_column     = 'id',
    out_model_file   = 'dt_model.txt',
    out_cm_file      = 'dt_cm_model.csv',

    clasifier_config = {
        'classifier': 'dt',
        'classifier.dt.max': '10'
    },
    verbose          = 0
):
    features_param = " ".join(features)

    clasifier_params = ''
    for k, v in clasifier_config.items():
        clasifier_params += f' -{k} {v}' 
        
    if verbose > 0:
        print_title(f'Training {clasifier_config["classifier"]} classifier')
        print(f'- SQL Path.............: {result_path(sql_file)}')
        print(f'- Stats Path...........: {result_path(stat_file)}')
        print(f'- Target...............: {label_column}')
        print(f'- Features.............: {features}')
        print(f'- Model Path...........: {result_path(out_model_file)}')
        print(f'- Confusion Matrix Path: {result_path(out_cm_file)}')
        print(f'- Clasifier config.....: {clasifier_params}\n\n')

    !time otbcli_TrainVectorClassifier \
        -io.vd             {result_path(sql_file)} \
        -io.stats          {result_path(stat_file)} \
        -feat              {features_param} \
        -io.out            {result_path(out_model_file)} \
        -io.confmatout     {result_path(out_cm_file)} \
        -cfield            {label_column} \
        {clasifier_params}

In [10]:
def band_math(
    raster_paths,
    name           = 'ndvi',
    formula        = '(im1b7-im1b3)/(im1b7+im1b3)',
    extension      = '.tif', 
    verbose        = 0
):
    out_path = lambda path: f'{path.split(extension)[0]}_{name}.tif'
    
    if verbose > 0:
        print_title(f'Calculate "{name} = {formula}" formula')

    outputs = []
    for in_raster_path in raster_paths:
        out_raster_path = out_path(in_raster_path)

        if verbose > 0:
            print(f'\n\n- In: {in_raster_path}\n- Out: {out_raster_path}')

        !time otbcli_BandMath \
            -il  {in_raster_path} \
            -out {out_raster_path} \
            -exp "{formula}"
        outputs.append(out_raster_path)
    return outputs

In [11]:
ALL_VEGETATION_INDEXES = [
    'Vegetation:NDVI',
    'Vegetation:TNDVI', 
    'Vegetation:RVI',
    'Vegetation:SAVI',
    'Vegetation:TSAVI', 
    'Vegetation:MSAVI', 
    'Vegetation:MSAVI2',
    'Vegetation:IPVI',
    'Vegetation:LAIFromNDVILog',
    'Vegetation:LAIFromReflLinear',
    'Vegetation:LAIFromNDVIFormo'
]

def compute_index(
    raster_paths,
    indexes     = ALL_VEGETATION_INDEXES,
    out_postfix = 'indexes',
    blue_band   = 1,
    green_band  = 1,
    red_band    = 1,
    nir_band    = 1,
    mir_band    = 1,
    extension   = '.tif', 
    verbose     = 0
):  
    out_path = lambda path: f'{path.split(extension)[0]}_{out_postfix}.tif'

    if verbose > 0:
        print_title(f'Calculate indexes: "{indexes}"')

    for in_raster_path in raster_paths:
        out_raster_path = out_path(in_raster_path)

        if verbose > 0:
            print(f'\n\n- In: {in_raster_path}\n- Out: {out_raster_path}')

        !time otbcli_RadiometricIndices      \
            -channels.blue  {blue_band}      \
            -channels.green {green_band}     \
            -channels.red   {red_band}       \
            -channels.nir   {nir_band}       \
            -channels.mir   {mir_band}       \
            -in             {in_raster_path} \
            -list           {" ".join(indexes)}        \
            -out            {out_raster_path}

In [12]:
def join_rasters(raster_paths, out_file, verbose = 0):
    if verbose > 0:
        print_title(f'Join rasters')
        print(f'- Input Rasters: {" ".join(raster_paths)}')
        print(f'- Output Raster: {" ".join(out_file)}')

    !time otbcli_ConcatenateImages \
        -il {' '.join(raster_paths)} \
        -out {out_file}

## Analisis

In [13]:
!mkdir -p {RESULT_PATH}

Listamos los archivos de datos:

In [14]:
!ls -la {DATA_PATH}/*shp 

-rw-rw-r--  1 adrian.marino  729499403   79252 Apr 24  2021 ../datasets/data/departamentos.shp
-rw-rw-r--  1 adrian.marino  729499403   13148 Apr 24  2021 ../datasets/data/verdad_campo.shp
-rw-r--r--  1 adrian.marino  729499403  182772 Jul  2 12:07 ../datasets/data/verdad_campo_aumentada.shp


Tenemos dos archivos, departamentes de buenos aires y la verdad de campo o poligonos labels. estos poligonos representan un label y cubro una parte de la superficie de las imagenes donde se encuentra esa misma clase.

Veamos que tablas contienen:

In [15]:
layer_info('departamentos')

INFO: Open of `../datasets/data/departamentos.shp'
      using driver `ESRI Shapefile' successful.
1: departamentos (3D Polygon)


In [16]:
layer_info(VERDAD_CAMPO)

INFO: Open of `../datasets/data/verdad_campo_aumentada.shp'
      using driver `ESRI Shapefile' successful.
1: verdad_campo_aumentada (Polygon)


In [17]:
layer_table_info(VERDAD_CAMPO, VERDAD_CAMPO)

INFO: Open of `../datasets/data/verdad_campo_aumentada.shp'
      using driver `ESRI Shapefile' successful.

Layer name: verdad_campo_aumentada
Metadata:
  DBF_DATE_LAST_UPDATE=2022-07-02
Geometry: Polygon
Feature Count: 466
Extent: (-65.082036, -35.296969) - (-62.290375, -33.823709)
Layer SRS WKT:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["latitude",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["longitude",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
in1: String (6.0)
id: Integer64 (10.0)
cultivo: String (10.0)


La capa o archivo de verdad de campo tiene la columna **cultivo** la cual es una columna categorica que tiene las clases en formato string.

In [18]:
layer_query(
    VERDAD_CAMPO, 
    f'SELECT * FROM {VERDAD_CAMPO} LIMIT 2'
)

INFO: Open of `../datasets/data/verdad_campo_aumentada.shp'
      using driver `ESRI Shapefile' successful.

Layer name: SELECT
Geometry: Polygon
Feature Count: 2
Extent: (-62.896600, -34.034258) - (-62.868281, -34.012151)
Layer SRS WKT:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["latitude",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["longitude",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Geometry Column = GEOMETRY
in1: String (0.0)
id: Integer64 (0.0)
cultivo: String (0.0)
OGRFeature(SELECT):0
  in1 (String) = 014084
  id (Integer64) = 2
  cultivo (String) = MAIZ
  POLYGON ((-62.8765996133436 -34.0221508898355,-62.8770890481807 -34.02

In [19]:
layer_query(
    VERDAD_CAMPO,
    f"""
    SELECT 
        cultivo  AS Cultivo,
        COUNT(*) AS Cantidad
    FROM
        {VERDAD_CAMPO}
    GROUP BY
        cultivo
    """
)

INFO: Open of `../datasets/data/verdad_campo_aumentada.shp'
      using driver `ESRI Shapefile' successful.

Layer name: SELECT
Geometry: None
Feature Count: 5
Layer SRS WKT:
(unknown)
Cultivo: String (0.0)
Cantidad: Integer (0.0)
OGRFeature(SELECT):0
  Cultivo (String) = ALFALFA
  Cantidad (Integer) = 2

OGRFeature(SELECT):1
  Cultivo (String) = CAMPONATUR
  Cantidad (Integer) = 18

OGRFeature(SELECT):2
  Cultivo (String) = GIRASOL
  Cantidad (Integer) = 32

OGRFeature(SELECT):3
  Cultivo (String) = MAIZ
  Cantidad (Integer) = 246

OGRFeature(SELECT):4
  Cultivo (String) = SOJA
  Cantidad (Integer) = 168



In [20]:
layer_query(
    VERDAD_CAMPO, 
    f"""
    SELECT 
        id       AS 'Codigo de cultivo',
        COUNT(*) AS Cantidad
    FROM
        {VERDAD_CAMPO}
    GROUP BY
        cultivo
    """
)

INFO: Open of `../datasets/data/verdad_campo_aumentada.shp'
      using driver `ESRI Shapefile' successful.

Layer name: SELECT
Geometry: None
Feature Count: 5
Layer SRS WKT:
(unknown)
Codigo de cultivo: Integer64 (0.0)
Cantidad: Integer (0.0)
OGRFeature(SELECT):0
  Codigo de cultivo (Integer64) = 10
  Cantidad (Integer) = 2

OGRFeature(SELECT):1
  Codigo de cultivo (Integer64) = 20
  Cantidad (Integer) = 18

OGRFeature(SELECT):2
  Codigo de cultivo (Integer64) = 5
  Cantidad (Integer) = 32

OGRFeature(SELECT):3
  Codigo de cultivo (Integer64) = 2
  Cantidad (Integer) = 246

OGRFeature(SELECT):4
  Codigo de cultivo (Integer64) = 1
  Cantidad (Integer) = 168



Las columnas **id** y **cultivo** sin intercambiables. Es decir que el id representa a cada tipo de cultivo.

### Estadisticas y merge de rasters

A continuacion veamos la cantidad de pixels en la imagen por cada clase. Seria la distribición de probabilidad discreta de la variable categorica **cultivo** para una imagen dada.

Ver: [PolygonClassStatistics](https://www.orfeo-toolbox.org/CookBook/Applications/app_PolygonClassStatistics.html)

In [21]:
class_statistics([raster_date_path('2020-10-01', '0000000000-0000000000')], verbose=1)



Generate class statistics...

- In Paths: ../datasets/images/2020-10-01/0000000000-0000000000.tif
- Vec Path: ../datasets/data/verdad_campo_aumentada.shp
- Field...: id
- Out Path: ../results/class_statistics.xml


zsh:1: command not found: otbcli_PolygonClassStatistics
otbcli_PolygonClassStatistics -in  -vec  -field id -out  > /dev/null  0.00s user 0.00s system 76% cpu 0.001 total
<?xml version="1.0" ?>
<GeneralStatistics>
    <Statistic name="samplesPerClass">
        <StatisticMap key="1" value="134" />
        <StatisticMap key="10" value="2" />
        <StatisticMap key="2" value="107" />
        <StatisticMap key="20" value="18" />
        <StatisticMap key="3" value="139" />
        <StatisticMap key="4" value="34" />
        <StatisticMap key="5" value="32" />


'../results/class_statistics.xml'

Vemos una frecuencia muy baja x clase. Esto se debe a que la imagen completa esta compuesta por las dos imagenes dentro de cada directorio de fecha. Por esta cuestión, primero debemos hacer un merge de ambas imagenes para luego calcular estadisticas, clasificar, etc... 

A continuación se hace mer de todos las imagenes(rasters) por fecha: 

In [ ]:
for date in DATES:
    source_files = [raster_date_path(date, f) for f in RASTER_FILES]
    target_file  =  raster_date_path(date, 'complete_raster')
    merge_rasters(source_files, target_file)

0

Ahora validemos si vemos diferencia en las frecuencias:

In [ ]:
class_statistics([raster_date_path('2020-10-01', '0000000000-0000000000')], verbose=1)
class_statistics([raster_date_path('2020-10-01', '0000000000-0000012544')], verbose=1)
class_statistics([raster_date_path('2020-10-01', 'complete_raster')], verbose=1)

In [ ]:
class_statistics([raster_date_path('2021-03-17', '0000000000-0000000000')], verbose=1)
class_statistics([raster_date_path('2021-03-17', '0000000000-0000012544')], verbose=1)
class_statistics([raster_date_path('2021-03-17', 'complete_raster')], verbose=1)

**Por que todas las imagenes tiene la misma cantidad de pixeles por clase?**

### Sampling

A continacion sampleamos una cantidad de pixels por clase. De esta forma podemos estratificar las obsercaciones por clase, y asi evitar el desbalanceo de las clases.

Ver: [SampleSelection](https://www.orfeo-toolbox.org/CookBook/Applications/app_SampleSelection.html)

In [25]:
for date in DATES:
    raster_paths = [raster_date_path(date, 'complete_raster')]

    class_stat_path = class_statistics(raster_paths, out_file = f'{date}_class_stat.xml', verbose = 1)

    sampling(
        raster_paths,
        class_stat_path = class_stat_path,
        out_rates_path  = f'{date}_rates.csv',
        out_sql_path    = f'{date}_samples.sqlite',
        verbose         = 1
    )



Generate class statistics...

- In Paths: ../datasets/images/2020-10-01/complete_raster.tif
- Vec Path: ../datasets/data/verdad_campo.shp
- Field...: id
- Out Path: ../results/2020-10-01_class_stat.xml


zsh:1: command not found: otbcli_PolygonClassStatistics
otbcli_PolygonClassStatistics -in  -vec ../datasets/data/verdad_campo.shp  id  0.00s user 0.00s system 72% cpu 0.002 total
<?xml version="1.0" ?>
<GeneralStatistics>
    <Statistic name="samplesPerClass">
        <StatisticMap key="1" value="134" />
        <StatisticMap key="10" value="2" />
        <StatisticMap key="2" value="107" />
        <StatisticMap key="20" value="18" />
        <StatisticMap key="3" value="139" />
        <StatisticMap key="4" value="34" />
        <StatisticMap key="5" value="32" />


Sampling...

- In Paths......: ../datasets/images/2020-10-01/complete_raster.tif
- Vec Path......: ../datasets/data/verdad_campo.shp
- Field.........: id
- Instats Path..: ../results/2020-10-01_class_stat.xml
- Strategy

In [26]:
sql.SQLiteClient.inline_tables_definition(
    path  = result_path('2020-12-01_samples.sqlite'),
    table = 'output'
)

['CREATE TABLE "output" ( "ogc_fid" INTEGER PRIMARY KEY AUTOINCREMENT, "GEOMETRY" BLOB, "in1" VARCHAR(6), "id" BIGINT, "cultivo" VARCHAR(10), "originfid" INTEGER, "band_0" FLOAT, "band_1" FLOAT, "band_2" FLOAT, "band_3" FLOAT, "band_4" FLOAT, "band_5" FLOAT, "band_6" FLOAT)']

In [27]:
sql.SQLiteClient.inline_query(
    path  = result_path('2020-12-01_samples.sqlite'),
    query = """
        SELECT 
            cultivo,
            COUNT(*) AS Cantidad
        FROM
            output
        GROUP BY
            cultivo
        ORDER BY
            Cantidad desc
    """
)

,cultivo,Cantidad
0,SOJA,4
1,MAIZ,4
2,GIRASOL,2
3,CAMPONATUR,2
4,ALFALFA,2


### Extracción de observaciones


En este paso, en base a una capa vectorial (sqlite) y un raster, se genera la tabla **output** dentro del archivos de base de datos sqlite, donde cada fila es un pixel del raster y cada columna es el valor del pixel en cada banda que contenida en el mismo (En nuestro caso, como el raster es una imagen generada con el satelite SENTINEL).

Ver: [SampleExtraction](https://www.orfeo-toolbox.org/CookBook/Applications/app_SampleExtraction.html)

In [28]:
for date in DATES:
    sample_extraction(
        raster_paths = [raster_date_path(date, 'complete_raster')],
        out_sql_file = f'{date}_samples.sqlite',
        verbose      = 1
    )



Sample extraction...

- In Paths......: ../datasets/images/2020-10-01/complete_raster.tif
- Vec SQL Path..: ../results/2020-10-01_samples.sqlite
- Field.........: id


zsh:1: command not found: otbcli_SampleExtraction
otbcli_SampleExtraction -in ../datasets/images/2020-10-01/complete_raster.tif  0.00s user 0.00s system 73% cpu 0.001 total


Sample extraction...

- In Paths......: ../datasets/images/2020-11-01/complete_raster.tif
- Vec SQL Path..: ../results/2020-11-01_samples.sqlite
- Field.........: id


zsh:1: command not found: otbcli_SampleExtraction
otbcli_SampleExtraction -in ../datasets/images/2020-11-01/complete_raster.tif  0.00s user 0.00s system 73% cpu 0.001 total


Sample extraction...

- In Paths......: ../datasets/images/2020-12-01/complete_raster.tif
- Vec SQL Path..: ../results/2020-12-01_samples.sqlite
- Field.........: id


zsh:1: command not found: otbcli_SampleExtraction
otbcli_SampleExtraction -in ../datasets/images/2020-12-01/complete_raster.tif  0.00s user 0.00

In [29]:
sql.SQLiteClient.inline_tables_definition(
    path  = result_path('2020-12-01_samples.sqlite'),
    table = 'output'
)

['CREATE TABLE "output" ( "ogc_fid" INTEGER PRIMARY KEY AUTOINCREMENT, "GEOMETRY" BLOB, "in1" VARCHAR(6), "id" BIGINT, "cultivo" VARCHAR(10), "originfid" INTEGER, "band_0" FLOAT, "band_1" FLOAT, "band_2" FLOAT, "band_3" FLOAT, "band_4" FLOAT, "band_5" FLOAT, "band_6" FLOAT)']

In [30]:
sql.SQLiteClient.inline_query(
    path  = result_path('2020-12-01_samples.sqlite'),
    query = """
    SELECT
        id, cultivo, band_0, band_1, band_2, band_3, band_4, band_5, band_6
    FROM
        output
    """
)

,id,cultivo,band_0,band_1,band_2,band_3,band_4,band_5,band_6
0,1,SOJA,0.11030,0.14470,0.20530,0.2969,0.31280,0.44280,0.36000
1,2,MAIZ,0.08325,0.11240,0.15930,0.2303,0.24250,0.32375,0.27785
2,3,MAIZ,0.05295,0.07570,0.08695,0.2649,0.27645,0.27160,0.19525
3,5,GIRASOL,0.06445,0.10185,0.09960,0.3870,0.39970,0.29215,0.19620
4,20,CAMPONATUR,0.07070,0.10540,0.12120,0.2734,0.29410,0.32350,0.25000
5,20,CAMPONATUR,0.11770,0.15900,0.20000,0.3115,0.33135,0.37915,0.29460
6,4,SOJA,0.09440,0.12720,0.17160,0.2770,0.30390,0.37730,0.32870
7,5,GIRASOL,0.07450,0.10380,0.12310,0.2787,0.29680,0.33140,0.26365
8,3,MAIZ,0.01480,0.04590,0.01800,0.4756,0.47240,0.18260,0.08500
9,2,MAIZ,0.10005,0.12400,0.17030,0.2441,0.25575,0.40510,0.33370


### ComputeImageStatistics


Ver: [ComputeImageStatistics](https://www.orfeo-toolbox.org/CookBook/Applications/app_ComputeImagesStatistics.html)

In [31]:
for date in DATES:
    compute_raster_statistics(
        raster_paths = [raster_date_path(date, 'complete_raster')],
        stat_file    = f'{date}_norm_raster_stat.xml',
        verbose      = 1
    )



Compute rasters statistics...

- Raster Paths: ../datasets/images/2020-10-01/complete_raster.tif
- Stats Path : ../results/2020-10-01_norm_raster_stat.xml


zsh:1: command not found: otbcli_ComputeImagesStatistics
otbcli_ComputeImagesStatistics -il  -out.xml   0.00s user 0.00s system 74% cpu 0.001 total


Compute rasters statistics...

- Raster Paths: ../datasets/images/2020-11-01/complete_raster.tif
- Stats Path : ../results/2020-11-01_norm_raster_stat.xml


zsh:1: command not found: otbcli_ComputeImagesStatistics
otbcli_ComputeImagesStatistics -il  -out.xml   0.00s user 0.00s system 75% cpu 0.001 total


Compute rasters statistics...

- Raster Paths: ../datasets/images/2020-12-01/complete_raster.tif
- Stats Path : ../results/2020-12-01_norm_raster_stat.xml


zsh:1: command not found: otbcli_ComputeImagesStatistics
otbcli_ComputeImagesStatistics -il  -out.xml   0.00s user 0.00s system 71% cpu 0.001 total


Compute rasters statistics...

- Raster Paths: ../datasets/images/2021-01-01/

## Clasificacion por pixels


Ver: [TrainVectorClassifier](https://www.orfeo-toolbox.org/CookBook/Applications/app_TrainVectorClassifier.html)

In [32]:
for date in DATES:
    train_clasifier(
        sql_file         = f'{date}_samples.sqlite',
        stat_file        = f'{date}_norm_raster_stat.xml',
        out_model_file   = f'{date}_dt_model.txt',
        out_cm_file      = f'{date}_dt_cm_model.csv',
        verbose          = 1,
        clasifier_config = {
            'classifier': 'dt',
            'classifier.dt.min': 1,
            'classifier.dt.max': 10
        }
    )



Training dt classifier...

- SQL Path.............: ../results/2020-10-01_samples.sqlite
- Stats Path...........: ../results/2020-10-01_norm_raster_stat.xml
- Target...............: id
- Features.............: ['band_0', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6']
- Model Path...........: ../results/2020-10-01_dt_model.txt
- Confusion Matrix Path: ../results/2020-10-01_dt_cm_model.csv
- Clasifier config.....:  -classifier dt -classifier.dt.min 1 -classifier.dt.max 10


zsh:1: command not found: otbcli_TrainVectorClassifier
otbcli_TrainVectorClassifier -io.vd ../results/2020-10-01_samples.sqlite       0.00s user 0.00s system 74% cpu 0.001 total


Training dt classifier...

- SQL Path.............: ../results/2020-11-01_samples.sqlite
- Stats Path...........: ../results/2020-11-01_norm_raster_stat.xml
- Target...............: id
- Features.............: ['band_0', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6']
- Model Path...........: ../results/2020-11-01_

## Join de campañas de cultivo y calculo de indices

Partiendo de los rasters(.tif) pertenecientes a cada campaña de cultivo (6 en total), calculamos el indice NDVI para cada uno de estos. Luego realizamos la acción **Concat**, la cual realiza un join de todos los rasters. Esto significa que, como resultado tendremos un unico raster con las columnas de los 6 rasters iniciales. Como filas tendremos el mismo numero de pixels que los rasters iniciales. Por ejemplo: dados dos rasters 1 y 2 los cuales tiene 10 filas(pixels) y 1 columna cada uno, al concatenarlos tenemos un unico raster de 10 filas y 2 columna. **Concat** es el análogo al aplicar JOIN en sql.

### 1. Primero calculamos lo indice que creamos necesarios. Estos seran utilizados como features en el paso de clasificación.

In [33]:
raster_paths = [raster_date_path(date, 'complete_raster') for date in DATES]
raster_paths

['../datasets/images/2020-10-01/complete_raster.tif',
 '../datasets/images/2020-11-01/complete_raster.tif',
 '../datasets/images/2020-12-01/complete_raster.tif',
 '../datasets/images/2021-01-01/complete_raster.tif',
 '../datasets/images/2021-02-20/complete_raster.tif',
 '../datasets/images/2021-03-17/complete_raster.tif']

<img src="../images/lansat2_bands.png" alt="LANSAT2 Bands" width="800">

**Nota**: Las bandas en los rasters comienzan desde 0 no desde 1.

Ver: [RadiometricIndices](https://www.orfeo-toolbox.org/CookBook/Applications/app_RadiometricIndices.html)

In [34]:
ALL_VEGETATION_INDEXES

['Vegetation:NDVI',
 'Vegetation:TNDVI',
 'Vegetation:RVI',
 'Vegetation:SAVI',
 'Vegetation:TSAVI',
 'Vegetation:MSAVI',
 'Vegetation:MSAVI2',
 'Vegetation:IPVI',
 'Vegetation:LAIFromNDVILog',
 'Vegetation:LAIFromReflLinear',
 'Vegetation:LAIFromNDVIFormo']

In [35]:
compute_index(
    raster_paths,
    indexes     = ALL_VEGETATION_INDEXES,
#    indexes     = 'Vegetation:NDVI',
    out_postfix = 'indexes',
    blue_band   = 1,
    green_band  = 2,
    red_band    = 3,
    nir_band    = 7,
    verbose     = 1
)



Calculate indexes: "['Vegetation:NDVI', 'Vegetation:TNDVI', 'Vegetation:RVI', 'Vegetation:SAVI', 'Vegetation:TSAVI', 'Vegetation:MSAVI', 'Vegetation:MSAVI2', 'Vegetation:IPVI', 'Vegetation:LAIFromNDVILog', 'Vegetation:LAIFromReflLinear', 'Vegetation:LAIFromNDVIFormo']"...



- In: ../datasets/images/2020-10-01/complete_raster.tif
- Out: ../datasets/images/2020-10-01/complete_raster_indexes.tif
zsh:1: command not found: otbcli_RadiometricIndices
otbcli_RadiometricIndices -channels.blue 1 -channels.green 2 -channels.red 3   0.00s user 0.00s system 73% cpu 0.001 total


- In: ../datasets/images/2020-11-01/complete_raster.tif
- Out: ../datasets/images/2020-11-01/complete_raster_indexes.tif
zsh:1: command not found: otbcli_RadiometricIndices
otbcli_RadiometricIndices -channels.blue 1 -channels.green 2 -channels.red 3   0.00s user 0.00s system 74% cpu 0.001 total


- In: ../datasets/images/2020-12-01/complete_raster.tif
- Out: ../datasets/images/2020-12-01/complete_raster_indexes.tif
zsh:1

Como resultado tenemos un nuevo raster por cada campaña de cultivo, cada uno contiene una columna por cada indice calculado.

In [36]:
!gdalinfo {raster_date_path('2020-10-01', 'complete_raster_indexes')}

ERROR 4: ../datasets/images/2020-10-01/complete_raster_indexes.tif: No such file or directory
gdalinfo failed - unable to open '../datasets/images/2020-10-01/complete_raster_indexes.tif'.


### 2. Join de rasters

En este paso vamos a realizar un join de todos los rasters del paso anterior.

In [37]:
join_rasters(
    raster_paths = [raster_date_path(date, 'complete_raster_indexes') for date in DATES],
    out_file     = result_path('complete_rasters_join'),
    verbose      = 1
)



Join rasters...

- Input Rasters: ../datasets/images/2020-10-01/complete_raster_indexes.tif ../datasets/images/2020-11-01/complete_raster_indexes.tif ../datasets/images/2020-12-01/complete_raster_indexes.tif ../datasets/images/2021-01-01/complete_raster_indexes.tif ../datasets/images/2021-02-20/complete_raster_indexes.tif ../datasets/images/2021-03-17/complete_raster_indexes.tif
- Output Raster: . . / r e s u l t s / c o m p l e t e _ r a s t e r s _ j o i n
zsh:1: command not found: otbcli_ConcatenateImages
otbcli_ConcatenateImages -il       -out ../results/complete_rasters_join  0.00s user 0.00s system 76% cpu 0.001 total


A continuación podemos observar que el raster consolidado tiene 66 Bandas. El formato raster agregar el nombre **Band** a sus columnas, ya que es la información mas común para este tipo de datos. En nuestro caso estas columnas representan los indices de cada raster de entrada calculado para por cada pixel. Como resultado tendremos un único raster o mapa con todos los indices calculado para cada raster inicial y los pixels del mapa como filas.

In [38]:
!gdalinfo  {result_path('complete_rasters_join.tif')}

ERROR 4: ../results/complete_rasters_join.tif: No such file or directory
gdalinfo failed - unable to open '../results/complete_rasters_join.tif'.


Cuanto pesa el mapa o raster consolidado? 

In [39]:
!du -h {result_path('complete_rasters_join.tif')}

du: ../results/complete_rasters_join.tif: No such file or directory


### 3. Clasificacion por pixes usando el raster de indices consolidado

In [40]:
class_stat_path = class_statistics(
    [result_path('complete_rasters_join.tif')], 
    out_file = 'complete_rasters_join_class_stat.xml', 
    verbose = 1
)



Generate class statistics...

- In Paths: ../results/complete_rasters_join.tif
- Vec Path: ../datasets/data/verdad_campo.shp
- Field...: id
- Out Path: ../results/complete_rasters_join_class_stat.xml


zsh:1: command not found: otbcli_PolygonClassStatistics
otbcli_PolygonClassStatistics -in ../results/complete_rasters_join.tif -vec    0.00s user 0.00s system 75% cpu 0.001 total
<?xml version="1.0" ?>
<GeneralStatistics>
    <Statistic name="samplesPerClass">
        <StatisticMap key="1" value="134" />
        <StatisticMap key="10" value="2" />
        <StatisticMap key="2" value="107" />
        <StatisticMap key="20" value="18" />
        <StatisticMap key="3" value="139" />
        <StatisticMap key="4" value="34" />
        <StatisticMap key="5" value="32" />


In [41]:
sampling(
    [result_path('complete_rasters_join.tif')],
    class_stat_path = class_stat_path,
    out_rates_path  = f'complete_rasters_join_rates.csv',
    out_sql_path    = f'complete_rasters_join_samples.sqlite',
    verbose         = 1,
    strategy        = 'total'
)



Sampling...

- In Paths......: ../results/complete_rasters_join.tif
- Vec Path......: ../datasets/data/verdad_campo.shp
- Field.........: id
- Instats Path..: ../results/complete_rasters_join_class_stat.xml
- Strategy......: total
- Out Rates Path: ../results/complete_rasters_join_rates.csv
- Out SQL Path..: ../results/complete_rasters_join_samples.sqlite


zsh:1: command not found: otbcli_SampleSelection
otbcli_SampleSelection -in ../results/complete_rasters_join.tif -vec  -instat  0.00s user 0.00s system 75% cpu 0.001 total


In [42]:
sql.SQLiteClient.inline_tables_definition(
    path  = result_path('complete_rasters_join_samples.sqlite'), 
    table = 'output'
)

['CREATE TABLE "output" ( "ogc_fid" INTEGER PRIMARY KEY AUTOINCREMENT, "GEOMETRY" BLOB, "in1" VARCHAR(6), "id" BIGINT, "cultivo" VARCHAR(10), "originfid" INTEGER, "band_0" FLOAT, "band_1" FLOAT, "band_2" FLOAT, "band_3" FLOAT, "band_4" FLOAT, "band_5" FLOAT, "band_6" FLOAT, "band_7" FLOAT, "band_8" FLOAT, "band_9" FLOAT, "band_10" FLOAT, "band_11" FLOAT, "band_12" FLOAT, "band_13" FLOAT, "band_14" FLOAT, "band_15" FLOAT, "band_16" FLOAT, "band_17" FLOAT, "band_18" FLOAT, "band_19" FLOAT, "band_20" FLOAT, "band_21" FLOAT, "band_22" FLOAT, "band_23" FLOAT, "band_24" FLOAT, "band_25" FLOAT, "band_26" FLOAT, "band_27" FLOAT, "band_28" FLOAT, "band_29" FLOAT, "band_30" FLOAT, "band_31" FLOAT, "band_32" FLOAT, "band_33" FLOAT, "band_34" FLOAT, "band_35" FLOAT, "band_36" FLOAT, "band_37" FLOAT, "band_38" FLOAT, "band_39" FLOAT, "band_40" FLOAT, "band_41" FLOAT, "band_42" FLOAT, "band_43" FLOAT, "band_44" FLOAT, "band_45" FLOAT, "band_46" FLOAT, "band_47" FLOAT, "band_48" FLOAT, "band_49" FLOA

In [43]:
sample_extraction(
    raster_paths = [result_path('complete_rasters_join.tif')],
    out_sql_file = f'complete_rasters_join_samples.sqlite',
    verbose      = 1
)



Sample extraction...

- In Paths......: ../results/complete_rasters_join.tif
- Vec SQL Path..: ../results/complete_rasters_join_samples.sqlite
- Field.........: id


zsh:1: command not found: otbcli_SampleExtraction
otbcli_SampleExtraction -in ../results/complete_rasters_join.tif -vec  -field  0.00s user 0.00s system 73% cpu 0.001 total


In [44]:
sql.SQLiteClient.inline_tables_definition(
    path  = result_path('complete_rasters_join_samples.sqlite'), 
    table = 'output'
)

['CREATE TABLE "output" ( "ogc_fid" INTEGER PRIMARY KEY AUTOINCREMENT, "GEOMETRY" BLOB, "in1" VARCHAR(6), "id" BIGINT, "cultivo" VARCHAR(10), "originfid" INTEGER, "band_0" FLOAT, "band_1" FLOAT, "band_2" FLOAT, "band_3" FLOAT, "band_4" FLOAT, "band_5" FLOAT, "band_6" FLOAT, "band_7" FLOAT, "band_8" FLOAT, "band_9" FLOAT, "band_10" FLOAT, "band_11" FLOAT, "band_12" FLOAT, "band_13" FLOAT, "band_14" FLOAT, "band_15" FLOAT, "band_16" FLOAT, "band_17" FLOAT, "band_18" FLOAT, "band_19" FLOAT, "band_20" FLOAT, "band_21" FLOAT, "band_22" FLOAT, "band_23" FLOAT, "band_24" FLOAT, "band_25" FLOAT, "band_26" FLOAT, "band_27" FLOAT, "band_28" FLOAT, "band_29" FLOAT, "band_30" FLOAT, "band_31" FLOAT, "band_32" FLOAT, "band_33" FLOAT, "band_34" FLOAT, "band_35" FLOAT, "band_36" FLOAT, "band_37" FLOAT, "band_38" FLOAT, "band_39" FLOAT, "band_40" FLOAT, "band_41" FLOAT, "band_42" FLOAT, "band_43" FLOAT, "band_44" FLOAT, "band_45" FLOAT, "band_46" FLOAT, "band_47" FLOAT, "band_48" FLOAT, "band_49" FLOA

In [45]:
sql.SQLiteClient.inline_query(
    path  = result_path('complete_rasters_join_samples.sqlite'),
    query = """
        SELECT 
            cultivo  AS 'Codigo de cultivo',
            COUNT(*) AS 'Cantidad de pixels (Solo aquellos que tiene verdad de campo)'
        FROM
            output
        GROUP BY
            cultivo
        ORDER BY
            'Cantidad de pixels (Solo aquellos que tiene verdad de campo)' desc
    """
)

,Codigo de cultivo,Cantidad de pixels (Solo aquellos que tiene verdad de campo)
0,SOJA,168
1,MAIZ,246
2,GIRASOL,32
3,CAMPONATUR,18
4,ALFALFA,2


In [46]:
compute_raster_statistics(
    raster_paths = [result_path('complete_rasters_join.tif')],
    stat_file    = f'complete_rasters_join_norm_raster_stat.xml',
    verbose      = 1
)



Compute rasters statistics...

- Raster Paths: ../results/complete_rasters_join.tif
- Stats Path : ../results/complete_rasters_join_norm_raster_stat.xml


zsh:1: command not found: otbcli_ComputeImagesStatistics
otbcli_ComputeImagesStatistics -il ../results/complete_rasters_join.tif    0.00s user 0.00s system 72% cpu 0.001 total


'../results/complete_rasters_join_norm_raster_stat.xml'

In [47]:
train_clasifier(
    sql_file         = f'complete_rasters_join_samples.sqlite',
    stat_file        = f'complete_rasters_join_norm_raster_stat.xml',
    out_model_file   = f'complete_rasters_join_rf_model.txt',
    out_cm_file      = f'complete_rasters_join_rf_cm_model.csv',
    verbose          = 1,
    features         = [f'band_{idx}' for idx in range(66)],
    clasifier_config = {
        'classifier': 'rf',
        'classifier.rf.min': 1,
        'classifier.rf.max': 10
    }
)



Training rf classifier...

- SQL Path.............: ../results/complete_rasters_join_samples.sqlite
- Stats Path...........: ../results/complete_rasters_join_norm_raster_stat.xml
- Target...............: id
- Features.............: ['band_0', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6', 'band_7', 'band_8', 'band_9', 'band_10', 'band_11', 'band_12', 'band_13', 'band_14', 'band_15', 'band_16', 'band_17', 'band_18', 'band_19', 'band_20', 'band_21', 'band_22', 'band_23', 'band_24', 'band_25', 'band_26', 'band_27', 'band_28', 'band_29', 'band_30', 'band_31', 'band_32', 'band_33', 'band_34', 'band_35', 'band_36', 'band_37', 'band_38', 'band_39', 'band_40', 'band_41', 'band_42', 'band_43', 'band_44', 'band_45', 'band_46', 'band_47', 'band_48', 'band_49', 'band_50', 'band_51', 'band_52', 'band_53', 'band_54', 'band_55', 'band_56', 'band_57', 'band_58', 'band_59', 'band_60', 'band_61', 'band_62', 'band_63', 'band_64', 'band_65']
- Model Path...........: ../results/complete_rast

## Construcción de dataset CSV

A partir de el conjunto completo de pixels que tienen verdad de campo construmimos un dataset en csv, renombrando las columnas para identificar a que indice y campaña de cultivo pertenecen.

In [48]:
df = sql.SQLiteClient.inline_query(
    path  = result_path('complete_rasters_join_samples.sqlite'), 
    query = 'SELECT * FROM output'
)

In [49]:
indexes_columns = flatten([[f'{value.replace("Vegetation:", "")}_{i+1}' for value in ALL_VEGETATION_INDEXES] for i in range(len(DATES))])

band_columns = np.array(list(filter(lambda v: 'band_' in v, df.columns)))

indexes_columns, band_columns

(array(['NDVI_1', 'TNDVI_1', 'RVI_1', 'SAVI_1', 'TSAVI_1', 'MSAVI_1',
        'MSAVI2_1', 'IPVI_1', 'LAIFromNDVILog_1', 'LAIFromReflLinear_1',
        'LAIFromNDVIFormo_1', 'NDVI_2', 'TNDVI_2', 'RVI_2', 'SAVI_2',
        'TSAVI_2', 'MSAVI_2', 'MSAVI2_2', 'IPVI_2', 'LAIFromNDVILog_2',
        'LAIFromReflLinear_2', 'LAIFromNDVIFormo_2', 'NDVI_3', 'TNDVI_3',
        'RVI_3', 'SAVI_3', 'TSAVI_3', 'MSAVI_3', 'MSAVI2_3', 'IPVI_3',
        'LAIFromNDVILog_3', 'LAIFromReflLinear_3', 'LAIFromNDVIFormo_3',
        'NDVI_4', 'TNDVI_4', 'RVI_4', 'SAVI_4', 'TSAVI_4', 'MSAVI_4',
        'MSAVI2_4', 'IPVI_4', 'LAIFromNDVILog_4', 'LAIFromReflLinear_4',
        'LAIFromNDVIFormo_4', 'NDVI_5', 'TNDVI_5', 'RVI_5', 'SAVI_5',
        'TSAVI_5', 'MSAVI_5', 'MSAVI2_5', 'IPVI_5', 'LAIFromNDVILog_5',
        'LAIFromReflLinear_5', 'LAIFromNDVIFormo_5', 'NDVI_6', 'TNDVI_6',
        'RVI_6', 'SAVI_6', 'TSAVI_6', 'MSAVI_6', 'MSAVI2_6', 'IPVI_6',
        'LAIFromNDVILog_6', 'LAIFromReflLinear_6', 'LAIFromNDVIForm

In [50]:
rename_def = {k:v for k,v in zip(band_columns, indexes_columns) }

### Target combinado

Concatenamos las columnas id y colvibo apra creat la columna target_combined.

In [51]:
df = df.rename(columns =rename_def)

df['target_combined'] = df["cultivo"] + '_' + df["id"].astype(str)

In [52]:
df.groupby(['target_combined']).size()

target_combined
ALFALFA_10         2
CAMPONATUR_20     18
GIRASOL_5         32
MAIZ_2           107
MAIZ_3           139
SOJA_1           134
SOJA_4            34
dtype: int64

### Target combinado solo maiz, soja  y others

Hacemso merge de todas catagorias expecto mais y soja en una nueva categoria OTHERS.

In [53]:
others = [ 'ALFALFA_10', 'CAMPONATUR_20', 'GIRASOL_5']
df['target_compined_maiz_soja_others'] = df['target_combined'].apply(lambda v: 'OTHER' if v in others else v)
df.groupby(['target_compined_maiz_soja_others']).size()

target_compined_maiz_soja_others
MAIZ_2    107
MAIZ_3    139
OTHER      52
SOJA_1    134
SOJA_4     34
dtype: int64

### Target maiz vs others

In [54]:
df['target_maiz_others'] = df['target_combined'].apply(lambda v: 1 if v in ['MAIZ_3', 'MAIZ_2'] else 0)
df.groupby(['target_maiz_others']).size()

target_maiz_others
0    220
1    246
dtype: int64

### Target maiz vs others

In [55]:
df['target_soja_others'] = df['target_combined'].apply(lambda v: 1 if v in ['SOJA_1', 'SOJA_4'] else 0)
df.groupby(['target_soja_others']).size()

target_soja_others
0    298
1    168
dtype: int64

## Agredamos columnas como min, max, mean, median, var

### Gardamos todos los datasets

In [56]:
df.to_csv( result_path('dataset.csv'), encoding='utf-8')

In [57]:
for col in indexes_columns:
    df[col] = (df[col] - df[col].mean()) / df[col].std() 

In [58]:
df.to_csv( result_path('dataset_norm.csv'), encoding='utf-8')

## Predicción

In [59]:
!otbcli_ImageClassifier \
    -in     {result_path('complete_rasters_join.tif')}  \
    -imstat {result_path('complete_rasters_join_norm_raster_stat.xml')} \
    -model  {result_path('complete_rasters_join_rf_model.txt')} \
    -out    {result_path('predict.tif')}

zsh:1: command not found: otbcli_ImageClassifier


In [60]:
result_path('complete_rasters_join.tif')

'../results/complete_rasters_join.tif'

In [61]:
result_path('predict.tif')

'../results/predict.tif'

In [62]:
plot.plot_raster(result_path('complete_rasters_join.tif'))

ERROR 4: ../results/complete_rasters_join.tif: No such file or directory


AttributeError: 'NoneType' object has no attribute 'GetRasterBand'

In [ ]:
!gdalinfo {raster_date_path('2020-11-01', 'complete_raster_ndvi')}

In [ ]:
!gdalinfo {result_path('predict.tif')}